In [2]:
import prehook

df_list = prehook.populate_dfs()

for df in df_list:
    table_name = df.get('table_name')
    file_id = df.get('file_id')
    message = f"Table name = {table_name} and file_id = {file_id}"
    print(message)


Table name = player and file_id = 1WJdoQ8oQ7qETcbu-OOgkd6t7UAAN59sx
Table name = winner and file_id = 1On6XlGXAdGk3vJkbdlM8A8xvQbG7_pRh
Table name = group and file_id = 1WJdoQ8oQ7qETcbu-OOgkd6t7UAAN59sx
Table name = team and file_id = 1XS9INg_SJ33Wp9MuTKi-PKCyKUJbgqH7
